In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("delta-lake-tutorial") \
    .config("spark.sql.session.timeZone", "Asia/Seoul") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [2]:
spark

In [16]:
df = spark.sql("select 1 as col1, 1L as col2")
df.printSchema()
df2 = df.withColumn("col3", expr("case when col2 is not null then 10 else col2 end"))
df2.printSchema()
df2.show()

root
 |-- col1: integer (nullable = false)
 |-- col2: long (nullable = false)

root
 |-- col1: integer (nullable = false)
 |-- col2: long (nullable = false)
 |-- col3: long (nullable = false)

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   1|  10|
+----+----+----+



In [38]:
print("a,b".find(","))

1


In [49]:
df = spark.sql("select 'ab' as col1")
def splitColumnA(col, sep):
    return split(col, sep)[0]
def splitColumnB(col, sep):
    return split(col, sep)[1]
df2 = df.withColumn("col2", splitColumnA(expr("col1"), ",")).withColumn("col3", splitColumnB(expr("col1"), ","))
df3 = df2.na.fill({"col3": "default_value"})
df3.show()

+----+----+-------------+
|col1|col2|         col3|
+----+----+-------------+
|  ab|  ab|default_value|
+----+----+-------------+



In [4]:
%load_ext sparksql_magic

In [5]:
%%sparksql

CREATE TABLE events (
  date DATE,
  eventId STRING,
  eventType STRING,
  data STRING)
USING DELTA

In [6]:
%%sparksql
CREATE TABLE foo
USING DELTA
LOCATION 'tmp/delta-table'

In [10]:
%%sparksql
select * from foo order by id asc

id
0
1
2
3
4
5
6
7
8
9
